# Phase 2: Item Content Embedding Generation

This notebook provides the setup for running the Item Content Encoder on Kaggle GPUs.

In [ ]:
# 1. Install dependencies
!pip install -q sentence-transformers pyarrow fastparquet

In [ ]:
# 2. Clone the repository (Replace with your actual repo URL)
# !git clone https://github.com/USER/REPO_NAME.git
# %cd REPO_NAME

# For now, we assume you uploaded the scripts to the current directory
import os
os.makedirs('experiments/data_v1', exist_ok=True)

### Step 3: Run Baseline Preprocessing (If not already done)
Ensure you have `data/` and `submit/` directories from the competition uploaded to Kaggle.

In [ ]:
!python3 src/preprocess.py --data-dir data --submit-dir submit --out-dir experiments/data_v1
!python3 src/data/items.py --data-dir data --out-dir experiments/data_v1

### Step 4: Run Content Embedding Inference
This will use the GPU to generate 768-D embeddings for all ~134k books.

In [ ]:
!python3 src/models/item_encoder.py --batch_size 128

In [ ]:
import pandas as pd
emb = pd.read_parquet('experiments/data_v1/item_embeddings.parquet')
print(f"Generated embeddings for {len(emb)} items.")
print(emb.head())